# Analysis Review Attitude in Python

This code is main edited by Enze Wang

# Attention

Please put four .csv file in data folder and put this code in main folder instead of code folder.

This code want calculate the number of positive review and negative review on different business and different nouns. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import itertools
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from nltk import FreqDist
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize, PorterStemmer, pos_tag 
from nltk.corpus import stopwords
from nltk.text import TextCollection
from nltk.corpus import wordnet
import nltk.tokenize as nt
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /Users/enze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/enze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/enze/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
important_word_tf_idf=pd.read_csv("data/importance_word_tf_idf.csv")

In [3]:
pizza_business_clean_review=pd.read_csv('data/pizza_business_clean_review.csv')

In [4]:
pizza_business_review=pd.read_csv('data/pizza_review.csv')

# Review Attitude Function

We want to build a function to test whether a small piece of review is positive and negative. There are two ways to do that. 

# SentimentIntensityAnalyzer

In [5]:
SIA=SentimentIntensityAnalyzer()

In [6]:
SIA.polarity_scores("It is not a bad apple")

{'neg': 0.0, 'neu': 0.584, 'pos': 0.416, 'compound': 0.431}

In [7]:
SIA.polarity_scores("The attress is too rude")

{'neg': 0.429, 'neu': 0.571, 'pos': 0.0, 'compound': -0.4588}

In [8]:
SIA.polarity_scores("the pizza here disgust me")

{'neg': 0.494, 'neu': 0.506, 'pos': 0.0, 'compound': -0.5994}

In [9]:
SIA.polarity_scores("wow, pizza here")

{'neg': 0.0, 'neu': 0.345, 'pos': 0.655, 'compound': 0.5859}

In [10]:
SIA=SentimentIntensityAnalyzer()
def Review_attitude_1(review):
    if SIA.polarity_scores(review)['compound']<-0.2:
        return(-1)
    elif SIA.polarity_scores(review)['compound']>0.2:
        return(1)
    else:
        return(0)


The first way is to use SentimentIntensityAnalyzer directly. The outputs of {'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'compound': -0.3412} show the negative, neutral, positive parts and the whole score of a piece of review. For example, it show that the "It is not a bad apple" is a positive review because the compound is positive. 

In our function, we use 0.2 and -0.2 as a threshold, review compound larger than 0.2 is positive review and smaller than -0.2 is negative review.

# Count Positive & Negative Words

Remember we have find all positive and negative words before, therefore we can count positive and negative words in a piece of words. 

In [11]:
Positive_Words=['good',
 'great',
 'like',
 'best',
 'delicious',
 'nice',
 'ha',
 'love',
 'definitely',
 'fresh',
 'friendly',
 'pretty',
 'better',
 'amazing',
 'favorite',
 'friend',
 'recommend',
 'sure',
 'special',
 'excellent',
 'awesome',
 'perfect',
 'super',
 'happy',
 'huge',
 'kind',
 'enjoyed',
 'sweet',
 'fantastic',
 'loved',
 'free',
 'party',
 'enjoy',
 'wonderful',
 'hand',
 'clean',
 'ready',
 'care',
 'fan',
 'wish',
 'perfectly',
 'fun',
 'liked',
 'cool',
 'yes',
 'impressed',
 'yummy',
 'helpful',
 'glad',
 'hope',
 'easy',
 'please',
 'honestly',
 'help',
 'pleasant',
 'thank',
 'giving',
 'wow',
 'generous',
 'outstanding',
 'credit',
 'truly',
 'thanks',
 'excited',
 'certainly',
 'beautiful',
 'interesting',
 'shared',
 'easily',
 'clearly',
 'treat',
 'true',
 'original',
 'fair',
 'perfection',
 'value',
 'comfortable',
 'fabulous',
 'save',
 'cute',
 'appreciate',
 'lovely',
 'pleased',
 'lover',
 'nicely',
 'welcoming',
 'hoping',
 'play',
 'lol',
 'satisfied',
 'pleasantly',
 'positive',
 'clear',
 'efficient',
 'welcome',
 'support',
 'rich',
 'goodness',
 'popular',
 'honest',
 'healthy',
 'agree',
 'strong',
 'smile',
 'enjoyable',
 'favor',
 'creative',
 'appreciated',
 'apologized',
 'bright',
 'enjoying',
 'lucky',
 'impressive',
 'heaven',
 'hopefully',
 'fit',
 'gift',
 'inviting',
 'courteous',
 'terrific',
 'superb',
 'trust',
 'funny',
 'bonus',
 'satisfying',
 'delightful',
 'relaxed',
 'luckily',
 'luck',
 'interested',
 'supreme',
 'sweetness',
 'opportunity',
 'winner',
 'highlight',
 'liking',
 'holiday',
 'sharing',
 'relaxing',
 'slaw',
 'comfort',
 'romantic',
 'accept',
 'win',
 'safe',
 'compliment',
 'improve',
 'nicer',
 'classy',
 'improved',
 'thrilled',
 'thankfully',
 'award',
 'complimentary',
 'pleasure',
 'charm',
 'acceptable',
 'promised',
 'greatest',
 'pride',
 'relax',
 'wise',
 'celebrate',
 'improvement',
 'neat',
 'promise',
 'heavenly',
 'respect',
 'charming',
 'fav',
 'gorgeous',
 'loyal',
 'played',
 'satisfy',
 'easier',
 'assured',
 'divine',
 'delight',
 'deliciousness',
 'welcomed',
 'happily',
 'saved',
 'fave',
 'importantly',
 'beautifully',
 'kudos',
 'exciting',
 'honor',
 'natural',
 'delectable',
 'benefit',
 'winning',
 'scrumptious',
 'haha',
 'cared',
 'superior',
 'reward',
 'greet',
 'ideal',
 'grand',
 'lively',
 'wonderfully',
 'cheer',
 'yay',
 'surely',
 'impress',
 'nicest',
 'worthy',
 'supporting',
 'hoped',
 'deliciously',
 'ensure',
 'amazed',
 'sunny',
 'curious',
 'greeting',
 'eager',
 'grace',
 'success',
 'gladly',
 'praise',
 'fond',
 'greater',
 'proud',
 'smart',
 'entertainment',
 'adorable',
 'adventurous',
 'laugh',
 'smiling',
 'desire',
 'gracious',
 'ability',
 'sexy',
 'convinced',
 'laughed',
 'interest',
 'generously',
 'happier',
 'celebrating',
 'attractive',
 'thanked',
 'loving',
 'adore',
 'enjoys',
 'liberty',
 'passion',
 'dear',
 'remarkable',
 'focused',
 'thankful',
 'joy',
 'delighted',
 'dedicated',
 'adventure',
 'successful',
 'laughing',
 'passionate',
 'encourage',
 'satisfaction',
 'kindly',
 'complimented',
 'entertaining',
 'pleasing',
 'truth',
 'satisfactory',
 'prize',
 'comforting',
 'friendliest',
 'engaging',
 'robust',
 'inspired',
 'good-',
 'enthusiastic',
 'competent',
 'grateful',
 'creativity',
 'expand',
 'calm',
 'cheerful',
 'wowed',
 'caring',
 'fiesta',
 'courtesy',
 'smiled',
 'bold',
 'fab',
 'festival',
 'responsible',
 'friendliness',
 'increase',
 'elegant',
 'kindness',
 'beauty',
 'diamond',
 'security',
 'comfortably',
 'glorious',
 'beloved',
 'ease',
 'brilliant',
 'warmly',
 'fortunate',
 'solution',
 'promising',
 'thoughtful',
 'alive',
 'confident',
 'friendlier',
 'energetic',
 'innovative',
 'hero',
 'safety',
 'hilarious',
 'resolve',
 'justice',
 'succeed',
 'excitement',
 'worthwhile',
 'loyalty',
 'masterpiece',
 'sophisticated',
 'celebrated',
 'vibrant',
 'magnificent',
 'honesty',
 'assure',
 'great-',
 'engaged',
 'entertained',
 'responsive',
 'hopeful',
 'attraction',
 'respectable',
 'sincerely',
 'miracle',
 'clever',
 'warmth',
 'fame',
 'graciously',
 'strength',
 'glory',
 'accepting',
 'peace',
 'profit',
 'united',
 'actively',
 'faith',
 'favorable',
 'enthusiasm',
 'happiness',
 'efficiently',
 'appreciation',
 'delightfully',
 'flawless',
 'pleaser',
 'respectful']


In [12]:
Negative_Words=['bad',
 'disappointed',
 'wrong',
 'terrible',
 'rude',
 'problem',
 'worst',
 'horrible',
 'poor',
 'unfortunately',
 'disappointing',
 'awful',
 'mistake',
 'die',
 'fired',
 'dirty',
 'lack',
 'sad',
 'crazy',
 'hate',
 'complain',
 'lost',
 'negative',
 'waste',
 'gross',
 'worse',
 'disappoint',
 'mess',
 'fire',
 'damn',
 'disappointment',
 'difficult',
 'odd',
 'shame',
 'suck',
 'doubt',
 'fault',
 'disgusting',
 'block',
 'bother',
 'ridiculous',
 'drunk',
 'sadly',
 'hell',
 'sick',
 'pita',
 'regret',
 'bomb',
 'crap',
 'bloody',
 'messed',
 'upset',
 'confused',
 'starving',
 'nasty',
 'annoying',
 'ignored',
 'trouble',
 'annoyed',
 'tired',
 'unprofessional',
 'shit',
 'shocked',
 'boring',
 'mad',
 'snob',
 'killer',
 'lazy',
 'crappy',
 'ruined',
 'dead',
 'unacceptable',
 'uncomfortable',
 'complained',
 'worry',
 'pain',
 'bitter',
 'blame',
 'disappoints',
 'warning',
 'broken',
 'ridiculously',
 'failed',
 'crushed',
 'skeptical',
 'angry',
 'bothered',
 'messy',
 'ugh',
 'terribly',
 'screwed',
 'wasted',
 'frustrating',
 'error',
 'freaking',
 'sucked',
 'rudely',
 'broke',
 'argue',
 'kill',
 'hurt',
 'inconvenience',
 'screaming',
 'fake',
 'lose',
 'ignore',
 'stupid',
 'fail',
 'fool',
 'bummer',
 'weak',
 'disaster',
 'unpleasant',
 'wtf',
 'fight',
 'ruin',
 'steal',
 'unfriendly',
 'lousy',
 'frustrated',
 'unfortunate',
 'accidentally',
 'lying',
 'delay',
 'forced',
 'chagrin',
 'insane',
 'jerk',
 'ill',
 'pathetic',
 'miserable',
 'sucker',
 'outrageous',
 'obnoxious',
 'idiot',
 'lame',
 'cry',
 'badly',
 'unhappy',
 'dislike',
 'pissed',
 'stingy',
 'death',
 'embarrassed',
 'accident',
 'fear',
 'criticism',
 'wasting',
 'clueless',
 'lied',
 'dump',
 'horrendous',
 'poisoning',
 'incompetent',
 'irritated',
 'loss',
 'beaten',
 'horribly',
 'shitty',
 'guilty',
 'hated',
 'racist',
 'disgusted',
 'liar',
 'harsh',
 'dissatisfied',
 'severely',
 'fuck',
 'blamed',
 'dumped',
 'unimpressed',
 'scream',
 'fails',
 'stress',
 'argued',
 'rudest',
 'stink',
 'struggle',
 'attack',
 'avoiding',
 'dumb',
 'dirt',
 'insult',
 'losing',
 'po',
 'ignoring',
 'cheat',
 'bothering',
 'desperate',
 'avoided',
 'denied',
 'arguing',
 'shock',
 'rotten',
 'critical',
 'fighting',
 'struggled',
 'ashamed',
 'piss',
 'disagree',
 'bizarre',
 'overload',
 'misleading',
 'dull',
 'war',
 'desperately',
 'embarrassing',
 'frustration',
 'chaotic',
 'arrogant',
 'rudeness',
 'chaos',
 'fallen',
 'drowned',
 'appalled',
 'unhealthy',
 'regretted',
 'offensive',
 'stressed',
 'died',
 'bitch',
 'killing',
 'suffer',
 'killed',
 'dick',
 'argument',
 'blind',
 'awkwardly',
 'loose',
 'annoyance',
 'disrespect',
 'failure',
 'inconvenient',
 'screamed',
 'slam',
 'scary',
 'dangerous',
 'ugly',
 'nonsense',
 'insulting',
 'freak',
 'scam',
 'violation',
 'horrid',
 'ghost',
 'shoot',
 'flop',
 'severe',
 'scared',
 'fooled',
 'horrific',
 'difficulty',
 'haunt',
 'rob',
 'unimpressive',
 'crime',
 'horror',
 'upsetting',
 'stolen',
 'distracted',
 'threatening',
 'unsatisfied',
 'gun',
 'neglected',
 'depressing',
 'jealous',
 'deceiving',
 'stressful']

In [13]:
def Review_attitude_2(review):
    Review_word=review.split(" ")
    Positive=0
    Negative=0
    for i in Review_word:
        if i in Positive_Words:
            Positive+=1
        elif i in Negative_Words:
            Negative+=1
    if Positive/(Positive+Negative+1)>0.5:
        return(1)
    elif Negative/(Positive+Negative+1)>0.5:
        return(-1)
    else:
        return(0)

In [14]:
Review_attitude_2("it is good")

0

# Test

We select some small piece reviews from our dataset and test the two abillities.

In [15]:
Review=["Parking is tough to get here so I don't get to go much",
        
        "Good burgers",
        
        "burgers here is not bad",
        
        "If you drink, it looks like they have a lot of different kinds of alcohol",
        
        "Many questionable characters here",
        
        "I do recommend getting their legendary fries at least once and I do recommend the egg burger",
       
        "Hot Dogs & Burgers were very good.",
        
        "And it's BYOB, so don't forget your wine",
        
        "Meatballs - pretty tasty themselves, but the red sauce is magical.", 

        "The pizza sauce is perfect",
        
        "There's a lounge downstairs, amazing beer selection",
        
        "The pizza is pretty good.",  
        
        "The wings were very good.",  
        
        "Service was good.",  
        
        "The prices seemed reasonable though I wasn't exactly sure of the portions.",
        
        "And their sandwiches are very good as well",
        
        "This is the worse cardboard covered cheese & sauce so called pizza I've ever had",
        
        "So far, the eggplant pizza is my favorite"]

Score1=[]

Score2=[]

for i in Review:
    Score1.append(Review_attitude_1(i))
    Score2.append(Review_attitude_2(i))
    

    
        
        

In [16]:
Score1

[0, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1]

In [17]:
Score2

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

As you can see, the Review_attitude_2 can hardly make a decision on the review positive negative because the review sometimes did not contains the positive or negative words. 

Therefore we take Review_attitude_1 as our review attitude function. 

# Review Cut

It is never a good idea to judge a whole review:

In [18]:
example_review=pizza_business_review['text'][0]
example_review

"Parking is tough to get here so I don't get to go much. I also get a little cautious sometimes as they frequently have multiple questionable looking characters.\n\nPro's:\n- Legendary fries that used to be super huge and gave you great value for your buck\n- Good burgers\n- If you drink, it looks like they have a lot of different kinds of alcohol\n\nCon's:\n- Customer Service? Hahaha. Not here. If you can get more than a fart out of one of the twin's working there, you are ahead of the pack.\n- Dirty, messy tables, crap everywhere.\n- Many questionable characters here. Someone once asked me if I wanted to buy stolen jewelry off them while I was eating. I'm being totally honest too.\n- Their french fry portions have gotten smaller. They used to need two baskets for a large fry and now only need one.\n- Really expensive hot dogs. \n- They have strict limits on ketchup. No I'm not joking. 2 for large fries, 1 for anything less.\n\nI do recommend getting their legendary fries at least onc

In [19]:
Review_attitude_1(example_review)

1

In [20]:
SIA.polarity_scores(example_review)

{'neg': 0.081, 'neu': 0.819, 'pos': 0.1, 'compound': 0.7033}

As you can see, the example review at least mention three kinds of service of this business, Park, Fries and Burger. But we have no idea the positive attitude is for which kinds of service. Therefore we must cut review into pieces. This step is different from the word cut before.

In [21]:
Noun_Variables_Translate=pd.read_csv('data/Noun_Variables_Translate.csv')
Important_Noun=Noun_Variables_Translate[Noun_Variables_Translate["Check"]==1]
Important_Noun=Important_Noun["En"].tolist()
Important_Noun

['pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'gluten',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gelato',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'pesto',
 'margherita',
 'chocolate',
 'pizzeria',
 'coff

In [22]:
pizza_business_review

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27
1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12
2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35
3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23
4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40
...,...,...,...,...,...,...,...,...,...,...
67164,67164,gcNRaJMDlSVs8YlqwpsRuw,_Rgn4SZT2wXyQnW9AD9Tog,nR3u-wQ99bop6IdGoU38KQ,1,1,0,0,I'm originally from the other side of Ohio and...,2019-09-21 00:24:03
67165,67165,yFxDOdiDb8HPaK1io81kiw,4r33dXcE1oYZxjONrhxTiA,yT1Ad3auuw1jAJEZHr7Yqw,5,1,1,1,"Yes, yes, YES!! We loved going to the place in...",2019-09-03 04:32:16
67166,67166,Z_xmoLgIMB4As90t0a5eBQ,Iy6HkAzFcpeU7GF0e1FRSA,R13VkcyUIU3xG2fRFbBvBA,4,3,3,3,Went here for a lunch meeting amidst s typical...,2019-12-11 16:18:26
67167,67167,CzYRFCnkbqOVb3WIMMrbfw,TP9Tz7gcAfK-BeT16qpZ2w,M-caugKzJ9qLOmCBfD9Nyg,1,1,0,0,My wife and I have eaten at La Brioche at leas...,2017-06-13 02:14:53


In [23]:
Clean_Review=[]
Review=pizza_business_review["text"]
for i in range(len(Review)):
    review=Review[i]
    review=review.lower()
    review=nltk.word_tokenize(review)
    review=[i for i in review]
    review=" ".join(review)
    Split_Review=review.replace("!",".")
    Split_Review=Split_Review.replace("?",".")
    Split_Review=Split_Review.replace(";",".")
    Split_Review=Split_Review.split(".")
    Clean_Review.append(Split_Review)

In [25]:
pizza_business_review["Clean_Review"]=Clean_Review

Therefore we cut review into several piece of review, split by some important symbols.

# Attention: Code running for long time below:

In [26]:
r,c=pizza_business_review.shape
c=len(Important_Noun)
Attitude=np.zeros([r,c])
for i in range(r):
    for j in range(c):
        Target=Important_Noun[j]
        Review=pizza_business_review["Clean_Review"][i]
        Target_Review=[]
        for Small_Review in Review:
            if Target in Small_Review:
                Target_Review.append(Small_Review)
        Target_Review=".".join(Target_Review)
        if any(c.isalpha() for c in Target_Review)==True:
            Attitude[i][j]=Review_attitude_1(Target_Review)
        else:
            Attitude[i][j]=np.nan

For example, when i=27 and j=1, our target word and review will be:

In [42]:
Target=Important_Noun[1]
Target

'food'

In [43]:
Review=pizza_business_review["Clean_Review"][27]
Review

['visiting from ca & arrived at the hotel on a sunday night ( yesterday ) ',
 " i did n't have a lot of options but the front desk suggested this place ",
 ' i got the shrimp sandwich & jalapeño poppers for delivery ',
 ' the food took a while & when i called the restaurant , the workers sounded baffled bc my order had been lost ',
 " it was n't until the manager ( matt , i believe ) came on the line to help me out ",
 ' i guess the driver had gotten lost & went to the wrong hotel ',
 " in the middle of my meal , i realized i did n't get a receipt so i called back & the manager was willing to drop it off on his way home ",
 ' i thought that was really sweet but i insisted he drop it off tomorrow ',
 ' i still woke up to my receipt & i must say , i was really impressed by him ',
 " the fact that i 'm from out of town clearly means i wo n't be a regular here , so he did n't have to go out of his way ",
 " it was unfortunate my food was late but unlike the servers who answered the phone &

Then our function will extract every small review include the target word "food" and combine it as a whole review. 

In [44]:
Target_Review=[]
for Small_Review in Review:
    if Target in Small_Review:
        Target_Review.append(Small_Review)
Target_Review=".".join(Target_Review)
Target_Review

" the food took a while & when i called the restaurant , the workers sounded baffled bc my order had been lost . it was unfortunate my food was late but unlike the servers who answered the phone & casually just said my order was n't placed , then handed the phone to the manager because they did n't want to deal with me , the manager going out of his way to make sure i was happy really stood out to me . also , the food really is delicious "

As you can see, the customers said a lot about "food" in his review, but the whole attitude is positive.

We check is it a real review or some space which has no means, if there is no valid review, just return np.nan

In [45]:
 any(c.isalpha() for c in Target_Review)==True

True

Then give a attitude toward this review and fill in -1,'n' or 1 into our matrix:

In [46]:
Review_attitude_1(Target_Review)

1

Therefore, the i and j entry of Attitude matrix is whether negative, neutral or positive of review i towards important words j. 

In [47]:
Attitude[27][1]

1.0

In [33]:
review_attitude=pd.DataFrame(Attitude, columns=Important_Noun)
review_attitude

,pizza,food,cheese,sauce,service,salad,restaurant,crust,chicken,beer,...,linguini,fennel,fryer,pea,coconut,mixture,peach,manicotti,hash,romaine
0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67164,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67165,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67166,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67167,NaN,1.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
review_attitude = pd.concat([pizza_business_review["review_id"],pizza_business_review["business_id"],pizza_business_review["stars"], review_attitude], axis=1)

In [35]:
review_attitude.to_csv("data/Word_Review_Matrix.csv")
review_attitude

,review_id,business_id,stars,pizza,food,cheese,sauce,service,salad,restaurant,...,linguini,fennel,fryer,pea,coconut,mixture,peach,manicotti,hash,romaine
0,hfXVxrlD6fr1kXfz6sR13Q,J9f-9Prw2YVM-fiZqv2fmQ,2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDQpLLFiAxeSxWI5ipwzBQ,lpel3pRFAzlow7u2TJMz4Q,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWmT90rP7wYo57dG1cyvww,Zja7ykq3icDzkN20XX6j1w,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0jduztItUhOh3FzKGjMxZQ,hr0vxeTzlHx6fTpSSmu2AA,5,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sxfmzhMvZ8LlI1x4fhNjKg,F4kjKxXXyn3e-BYVb1mWcg,4,1.0,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67164,gcNRaJMDlSVs8YlqwpsRuw,nR3u-wQ99bop6IdGoU38KQ,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67165,yFxDOdiDb8HPaK1io81kiw,yT1Ad3auuw1jAJEZHr7Yqw,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67166,Z_xmoLgIMB4As90t0a5eBQ,R13VkcyUIU3xG2fRFbBvBA,4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67167,CzYRFCnkbqOVb3WIMMrbfw,M-caugKzJ9qLOmCBfD9Nyg,1,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
